In [ ]:
def make_diagram_panel(baseline_mean, baseline_std):
    bm, bs = baseline_mean, baseline_std

    # Plot mean +/- std for the PtTFPP channel of the dye by itself. 
    plt.axhspan(bm - bs, bm + bs, color=greys[1], alpha=0.7)
    plt.axhline(bm, color=greys[2], lw=3, label='sensor alone')

    # Example where there is spectral overlap between the dye and the chemical.
    # Fluorescence should increase linearly with the concentration of the molecule
    slope = 20 # AU / uM of chemical
    concs = 500 / np.power(2, np.arange(0,6))
    expected_fluor = slope * concs + bm

    # Sample triplicate measurements from this curve. 
    triplicate_expectation = np.vstack([expected_fluor]*3)
    triplicate_measurement = np.random.normal(triplicate_expectation, scale=baseline_std)
    sim_measurement_df = pd.DataFrame(triplicate_measurement.T, columns='1,2,3'.split(','))
    sim_measurement_df['conc'] = concs
    long_sim_measurement_df = sim_measurement_df.melt(var_name='replicate', id_vars='conc')

    sns.lineplot(data=long_sim_measurement_df, x='conc', y='value', marker='d', color=blues[0],
                 label='spectral overlap')

    # Example where quenching is the only effect.
    # Modeled using Stern-Volmer eqn for intensity If = I0 / (1 + K_SV*[quencher]).
    KSV = 5e-3 # fluorescence / uM
    expected_fluor = bm / (1+KSV*concs)
    triplicate_expectation = np.vstack([expected_fluor]*3)
    triplicate_measurement = np.random.normal(triplicate_expectation, scale=baseline_std)
    sim_measurement_df = pd.DataFrame(triplicate_measurement.T, columns='1,2,3'.split(','))
    sim_measurement_df['conc'] = concs
    long_sim_measurement_df = sim_measurement_df.melt(var_name='replicate', id_vars='conc')

    sns.lineplot(data=long_sim_measurement_df, x='conc', y='value', marker='d',
                 color=sns.xkcd_rgb['dusty red'],
                 label='quenching')

    # Example where quenching and spectral overlap are both in play. 
    expected_fluor = bm / (1+KSV*concs) + slope * concs
    triplicate_expectation = np.vstack([expected_fluor]*3)
    triplicate_measurement = np.random.normal(triplicate_expectation, scale=baseline_std)
    sim_measurement_df = pd.DataFrame(triplicate_measurement.T, columns='1,2,3'.split(','))
    sim_measurement_df['conc'] = concs
    long_sim_measurement_df = sim_measurement_df.melt(var_name='replicate', id_vars='conc')

    sns.lineplot(data=long_sim_measurement_df, x='conc', y='value', marker='d',
                 color=sns.xkcd_rgb['dusty purple'],
                 label='both')